# Trabajo Integrador: DuckDB

## Instalacion

In [1]:
%pip install duckdb --upgrade


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Creando conexión

Nos permite establecer una conexión a una base de datos, por defecto, si no especificamos su nombre, la base de datos no persistirá y operará en memoria, por lo tanto no se almacenarán las tablas creadas. Trabajaremos en memoria ya que consideramos que es el fuerte de DUCKDB y su análisis es el pertinente de este trabajo de investigación. 

In [31]:
import duckdb as db

database = db.connect(database=":memory:")


##### Prueba con el dataset. En PostgreSQL almacenaremos archivos de órdenes y sus pagos. En AWS almacenaremos los productos y su categoría. El resto de archivos será almacenado de manera local, algunos en CSV y otros en PARQUET. Esta información se encuentra representada meidante un esquema en la documentación adjunta

In [40]:
%pip install -q kagglehub        
import kagglehub, shutil, pathlib

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

local_csv_files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_sellers_dataset.csv",
]

for file_name in local_csv_files:
    shutil.copy(f"{path}/{file_name}", f"dataset/{file_name}")

for file_name in local_csv_files:
    table = pathlib.Path(f"dataset/{file_name}").stem
    print(table)         
    database.execute(f"""
        CREATE OR REPLACE TABLE {table} AS
        SELECT * FROM read_csv_auto('dataset/{file_name}');
    """)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
olist_customers_dataset
olist_geolocation_dataset
olist_order_items_dataset
olist_order_reviews_dataset
olist_sellers_dataset


#### Conexión con AWS S3

In [4]:
%pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**NOTA:** Para utilizar AWS S3 es necesario setear las credenciales para acceder al bucket, para eso debemos crear un user en la IAM de AWS, asignarle permisos y finalmente crear las claves de acceso para este usuario. Luego, estas credenciales son obtenidas desde un .env.
Es importante que la región del bucket y del usuario sean la misma.

In [38]:
from dotenv import load_dotenv
load_dotenv()  

import os

database.sql("INSTALL httpfs; LOAD httpfs;")

database.sql(f"""
SET s3_region='{os.getenv("AWS_REGION")}';
SET s3_access_key_id='{os.getenv("AWS_ACCESS_KEY_ID")}';
SET s3_secret_access_key='{os.getenv("AWS_SECRET_ACCESS_KEY")}';
""")

In [39]:
database.sql("""
SELECT * FROM read_csv_auto('s3://ti-quadrelli-ribarov/olist_products_dataset.csv') LIMIT 5
""")

┌──────────────────────────────────┬───────────────────────┬─────────────────────┬────────────────────────────┬────────────────────┬──────────────────┬───────────────────┬───────────────────┬──────────────────┐
│            product_id            │ product_category_name │ product_name_lenght │ product_description_lenght │ product_photos_qty │ product_weight_g │ product_length_cm │ product_height_cm │ product_width_cm │
│             varchar              │        varchar        │        int64        │           int64            │       int64        │      int64       │       int64       │       int64       │      int64       │
├──────────────────────────────────┼───────────────────────┼─────────────────────┼────────────────────────────┼────────────────────┼──────────────────┼───────────────────┼───────────────────┼──────────────────┤
│ 1e9e8ef04dbcff4541ed26657ea517e5 │ perfumaria            │                  40 │                        287 │                  1 │              225 │     

### Exploracion del dataset

In [ ]:
# Creación de la tabla en memoria
database.sql("CREATE TABLE customers AS SELECT * FROM 'olist_customers_dataset.csv'")


In [34]:

# Consulta a la tabla en memoria

table_description = database.sql("DESCRIBE customers")

print(table_description)

first_ten = database.sql("SELECT * FROM customers LIMIT 10")

print(first_ten)

CatalogException: Catalog Error: Table with name customers does not exist!
Did you mean "olist_customers_dataset"?